In [1]:
import os 
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

logger.info("Application started successfully.")

2025-06-13 02:48:36,846 - INFO - Application started successfully.


In [2]:
# load environment variables

storage_account = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
access_key = os.environ["AZURE_STORAGE_ACCOUNT_ACCESS_KEY"]
NESSIE_URI = os.environ['NESSIE_URI']
REF = "main"
FULL_PATH_TO_WAREHOUSE = os.environ['WAREHOUSE']
AWS_S3_ENDPOINT = os.environ["AWS_S3_ENDPOINT"]
AWS_ACCESS_KEY = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_KEY  = os.environ['AWS_SECRET_ACCESS_KEY']

In [3]:
# set pyspark configuration
from pyspark import SparkConf

conf = SparkConf() \
    .setAppName("process_bronze_adventureWorks") \
    .setMaster("local[*]") \
    .set("spark.executor.memory", "4g") \
    .set("spark.driver.memory", "2g") \

# set MinIO config
conf.set("fs.s3a.access.key", AWS_ACCESS_KEY) 
conf.set("fs.s3a.secret.key", AWS_SECRET_KEY) 
conf.set("fs.s3a.endpoint", AWS_S3_ENDPOINT) 
conf.set("fs.s3a.connection.ssl.enabled", "false") 
conf.set("fs.s3a.path.style.access", "true") 

# conf.set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
# conf.set("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
# conf.set("spark.sql.catalog.nessie.warehouse", FULL_PATH_TO_WAREHOUSE)
# conf.set("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
# conf.set("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
# conf.set("spark.sql.catalog.nessie.uri", NESSIE_URI)
# conf.set("spark.sql.catalog.nessie.ref", REF)
# conf.set("spark.sql.catalog.nessie.cache-enabled", "false")

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

25/06/13 02:48:37 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


# Process sales header

In [10]:
spark.sql("DROP NAMESPACE IF EXISTS nessie.bronze")
spark.sql("CREATE NAMESPACE nessie.bronze")
spark.sql("SHOW NAMESPACES in nessie").show()


+---------+
|namespace|
+---------+
|   bronze|
+---------+



In [14]:
# define list of data
source_list = [
    {
        "path": "s3a://seed/adventureWorks/currency_rate.csv",
        "iceberg_table": "nessie.bronze.currency_rate",
        "enabled": True
    },
    {
        "path": "s3a://seed/adventureWorks/currency.csv",
        "iceberg_table": "nessie.bronze.currency",
        "enabled": True
    },
    {
        "path": "s3a://seed/adventureWorks/customer.csv",
        "iceberg_table": "nessie.bronze.customer",
        "enabled": True
    },
    {
        "path": "s3a://seed/adventureWorks/product.csv",
        "iceberg_table": "nessie.bronze.product",
        "enabled": True
    },
    {
        "path": "s3a://seed/adventureWorks/sales.csv",
        "iceberg_table": "nessie.bronze.sales",
        "enabled": True
    }
]

In [17]:
for item in [item for item in source_list if item["enabled"]]:
    # Read CSV from MinIO
    logger.info(f'read csv from bucket: {item["path"]}')
    df = spark.read.option("header", "true") \
        .csv(item["path"])
    
    num_columns, num_rows = len(df.columns), df.count()

    # Write to Nessie table using createOrReplace
    logger.info(f'writing data ({num_rows} rows, {num_columns} columns) into: {item["iceberg_table"]}')
    df.writeTo(item["iceberg_table"]).createOrReplace() 

    logger.info(f'Writing complete: {item["iceberg_table"]}')

2025-06-13 03:05:55,313 - INFO - read csv from bucket: s3a://seed/adventureWorks/currency_rate.csv
2025-06-13 03:05:55,998 - INFO - writing data (14263 rows, 5 columns) into: nessie.bronze.currency_rate
2025-06-13 03:05:56,356 - INFO - Writing complete: nessie.bronze.currency_rate
2025-06-13 03:05:56,357 - INFO - read csv from bucket: s3a://seed/adventureWorks/currency.csv
2025-06-13 03:05:56,566 - INFO - writing data (105 rows, 4 columns) into: nessie.bronze.currency
2025-06-13 03:05:56,819 - INFO - Writing complete: nessie.bronze.currency
2025-06-13 03:05:56,819 - INFO - read csv from bucket: s3a://seed/adventureWorks/customer.csv
2025-06-13 03:05:57,136 - INFO - writing data (18484 rows, 17 columns) into: nessie.bronze.customer
2025-06-13 03:05:57,554 - INFO - Writing complete: nessie.bronze.customer
2025-06-13 03:05:57,555 - INFO - read csv from bucket: s3a://seed/adventureWorks/product.csv
2025-06-13 03:05:57,771 - INFO - writing data (295 rows, 10 columns) into: nessie.bronze.pro